# 01. Ingeniería de Datos (City Football Group Engine)

Este notebook es el motor del proyecto. Su misión es transformar datos crudos en KPIs profesionales.

**Fases:**
1. **Carga Viva**: Descarga de datos en tiempo real de la temporada actual.
2. **Limpieza y Unificación**: Cruce de datos de partidos con Valor de Mercado (Transfermarkt) y FIFA (SoFIFA).
3. **The City Engine**: Generación de métricas avanzadas (xG Proxy, Presión, Dominio).
4. **Rating Engines**: Cálculo de Elo, Glicko-2 y Dixon-Coles partido a partido.
5. **Exportación**: `df_final_features.csv`.

In [33]:
import pandas as pd
import numpy as np
import requests
import io
import json
import os
from datetime import datetime, timedelta
from difflib import get_close_matches

# Configuración
DATA_DIR = 'data'
OUTPUT_FILE = 'df_final_features.csv'
CURRENT_SEASON_URL = 'https://www.football-data.co.uk/mmz4281/2425/SP1.csv'

# Asegurar directorio
os.makedirs(DATA_DIR, exist_ok=True)

## 1. Función `fetch_live_data` (Sincronización Real)

In [34]:
def fetch_live_data():
    """
    Descarga el CSV de la temporada actual y lo guarda localmente.
    Si falla, usa el archivo local existente.
    """
    print(f"🔄 Intentando descargar datos en vivo de: {CURRENT_SEASON_URL}...")
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        r = requests.get(CURRENT_SEASON_URL, headers=headers, timeout=10)
        if r.status_code == 200:
            # Guardar como SP1_2425.csv (asumiendo temporada actual)
            # Nota: Ajustar el nombre del archivo según la temporada si es necesario automágicamente
            with open(os.path.join(DATA_DIR, 'SP1_2425.csv'), 'wb') as f:
                f.write(r.content)
            print("✅ Datos descargados y actualizados correctamente.")
            return True
        else:
            print(f"⚠️ Error al descargar: Status {r.status_code}")
            return False
    except Exception as e:
        print(f"❌ Excepción al descargar: {e}")
        return False

# Ejecutar sincronización al inicio
fetch_live_data()

🔄 Intentando descargar datos en vivo de: https://www.football-data.co.uk/mmz4281/2425/SP1.csv...
✅ Datos descargados y actualizados correctamente.


True

## 2. Carga y Unificación de Datos Históricos

In [35]:
# Cargar archivos JSON auxiliares
with open(os.path.join(DATA_DIR, 'sofifa_history.json'), 'r', encoding='utf-8') as f:
    sofifa_data = json.load(f)

with open(os.path.join(DATA_DIR, 'transfermarkt_history.json'), 'r', encoding='utf-8') as f:
    tm_data = json.load(f)

# Lista de temporadas a cargar (ajustar rango según disponibilidad real)
# Ejemplo: 2010 a 2025
seasons = []
dfs = []

for year in range(10, 26): # 10-11 a 24-25
    s_str = f"{year:02d}{(year+1)%100:02d}"
    filename = f"SP1_{s_str}.csv"
    path = os.path.join(DATA_DIR, filename)
    
    if os.path.exists(path):
        try:
            df_temp = pd.read_csv(path, encoding='latin1', on_bad_lines='skip') # 'encoding' suele ser necesario para football-data
            df_temp['Season'] = 2000 + year
            dfs.append(df_temp)
            seasons.append(2000 + year)
        except Exception as e:
            print(f"Error leyendo {filename}: {e}")

df_raw = pd.concat(dfs, ignore_index=True)
df_raw['Date'] = pd.to_datetime(df_raw['Date'], dayfirst=True, errors='coerce')
df_raw = df_raw.sort_values('Date').reset_index(drop=True)

print(f"Total partidos cargados: {len(df_raw)}")

Total partidos cargados: 5700


C:\Users\emili\AppData\Local\Temp\ipykernel_11480\1993894176.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_raw['Date'] = pd.to_datetime(df_raw['Date'], dayfirst=True, errors='coerce')


### Normalización de Nombres de Equipos

In [36]:
# Mapa manual para casos difíciles, el resto usará fuzzy match o directo
team_map = {
    'Ath Bilbao': 'Athletic Club',
    'Atletico Madrid': 'Atlético Madrid',
    'Espanol': 'RCD Espanyol',
    'Sp Gijon': 'Sporting Gijón',
    'Real Madrid': 'Real Madrid',
    'Barcelona': 'FC Barcelona',
    'Valencia': 'Valencia CF',
    'Sevilla': 'Sevilla FC',
    'Betis': 'Real Betis Balompié',
    'Sociedad': 'Real Sociedad',
    'La Coruna': 'Deportivo de La Coruña',
    'Celta': 'RC Celta de Vigo',
    'Villarreal': 'Villarreal CF',
    'Mallorca': 'RCD Mallorca',
    'Osasuna': 'CA Osasuna',
    'Alaves': 'Deportivo Alavés',
    'Vallecano': 'Rayo Vallecano',
    'Getafe': 'Getafe CF',
    'Levante': 'Levante UD',
    'Girona': 'Girona FC',
    'Las Palmas': 'UD Las Palmas',
    'Leganes': 'CD Leganés',
    'Almeria': 'UD Almería',
    'Granada': 'Granada CF',
    'Elche': 'Elche CF',
    'Valladolid': 'Real Valladolid CF',
    'Eibar': 'SD Eibar',
    'Huesca': 'SD Huesca',
    'Cadiz': 'Cádiz CF',
    'Sp Gijon': 'Sporting de Gijón'
}

def standardize_team(name):
    return team_map.get(name, name)

df_raw['HomeTeam'] = df_raw['HomeTeam'].apply(standardize_team)
df_raw['AwayTeam'] = df_raw['AwayTeam'].apply(standardize_team)

## 3. Generador de Proxies ("The City Engine")

In [37]:
# Funciones Auxiliares para Datos Externos
def get_team_metadata(team_name, season_year, source_data, default_val=None, value_key='value'):
    year_str = str(season_year)
    if year_str not in source_data:
        return default_val
    
    candidates = source_data[year_str]
    # Buscar coincidencia exacta o cercana
    # Optimizacion rapida: buscar substring
    for item in candidates:
        t = item['team']
        if team_name in t or t in team_name:
            if value_key == 'ova':
                return int(item['ova'])
            elif value_key == 'market':
                # Limpiar valor "€100m" -> 100
                val_str = item.get('value', '0')
                val_clean = val_str.replace('€', '').replace('m', '').replace('Th', '/1000').strip()
                
                # FIX: Handle Billions (bn)
                if 'bn' in val_clean:
                    try:
                        return float(val_clean.replace('bn', '')) * 1000
                    except:
                        return default_val
                        
                try:
                    if '/1000' in val_clean:
                        return float(val_clean.replace('/1000', '')) / 1000
                    return float(val_clean)
                except:
                    return default_val
    return default_val

# Feature Engineering Vectorizado
df = df_raw.copy()

# 1. Proxy xG (Expected Goals)
# Formula: (Tiros * 0.09) + (TirosPuerta * 0.29)
# Usando HS (Home Shots) y HST (Home Shots on Target)
df['Home_xG_Proxy'] = (df['HS'] * 0.09) + (df['HST'] * 0.29)
df['Away_xG_Proxy'] = (df['AS'] * 0.09) + (df['AST'] * 0.29)

# 2. Proxy Presion
# Posesion no siempre esta disponible en footbal-data antigua (antes de 2018 aprox)
# Si falta, estimamos posesion basada en cuotas (equipos favoritos suelen tener mas posesion)
# Estimacion simple: 50% si no hay dato
df['Home_Possession_Est'] = 0.50 # Placeholder si no tenemos dato real
df['Away_Possession_Est'] = 0.50

df['Home_Pressure'] = (df['HF'] + df['HY'] + df['HR']) / df['Away_Possession_Est'] # Faltas del local sobre posesion rival
df['Away_Pressure'] = (df['AF'] + df['AY'] + df['AR']) / df['Home_Possession_Est']

# 3. Proxy Dominio
# % Corners en campo rival
total_corners = df['HC'] + df['AC']
df['Home_Dominance'] = np.where(total_corners > 0, df['HC'] / total_corners, 0.5)
df['Away_Dominance'] = np.where(total_corners > 0, df['AC'] / total_corners, 0.5)

# 4. Fatiga Real
# Dias desde el ultimo partido
df['Home_Rest_Days'] = 7.0 # Default
df['Away_Rest_Days'] = 7.0

last_date = {}

for idx, row in df.iterrows():
    date = row['Date']
    home = row['HomeTeam']
    away = row['AwayTeam']
    
    if home in last_date:
        delta = (date - last_date[home]).days
        df.at[idx, 'Home_Rest_Days'] = min(delta, 30) # Cap en 30 dias
    
    if away in last_date:
        delta = (date - last_date[away]).days
        df.at[idx, 'Away_Rest_Days'] = min(delta, 30)
        
    last_date[home] = date
    last_date[away] = date

# 5. Enriquecimiento con FIFA y Valor de Mercado
# Esto lo haremos fila a fila porque depende del año de la temporada
home_fifa_ovas = []
away_fifa_ovas = []
home_market_vals = []
away_market_vals = []

for idx, row in df.iterrows():
    season = row['Season']
    h_ova = get_team_metadata(row['HomeTeam'], season, sofifa_data, default_val=75, value_key='ova')
    a_ova = get_team_metadata(row['AwayTeam'], season, sofifa_data, default_val=75, value_key='ova')
    h_val = get_team_metadata(row['HomeTeam'], season, tm_data, default_val=10.0, value_key='market')
    a_val = get_team_metadata(row['AwayTeam'], season, tm_data, default_val=10.0, value_key='market')
    
    home_fifa_ovas.append(h_ova)
    away_fifa_ovas.append(a_ova)
    home_market_vals.append(h_val)
    away_market_vals.append(a_val)

df['Home_FIFA_Ova'] = home_fifa_ovas
df['Away_FIFA_Ova'] = away_fifa_ovas
df['Home_Market_Value'] = home_market_vals
df['Away_Market_Value'] = away_market_vals

## 4. Rating Engines (Elo, Glicko-2, Dixon-Coles)

In [38]:
# Implementacion Vectorizada/Iterativa de Elo
# Para Dixon-Coles y Glicko simplificados usaremos versiones iterativas rapidas

elo_ratings = {team: 1500 for team in df['HomeTeam'].unique()}
elo_k = 20

defense_ratings = {team: 1.0 for team in df['HomeTeam'].unique()}
attack_ratings = {team: 1.0 for team in df['HomeTeam'].unique()}

home_elo_series = []
away_elo_series = []
home_att_series = []
away_att_series = []
home_def_series = []
away_def_series = []

def get_result_score(ftr):
    if ftr == 'H': return 1.0
    if ftr == 'D': return 0.5
    return 0.0

# Parametros Dixon Coles Simple (Actualizacion progresiva)
dc_lr = 0.01 # Learning rate

for idx, row in df.iterrows():
    h_team = row['HomeTeam']
    a_team = row['AwayTeam']
    
    # --- Antes del partido: Guardar Ratings ---
    h_elo = elo_ratings.get(h_team, 1500)
    a_elo = elo_ratings.get(a_team, 1500)
    
    h_att = attack_ratings.get(h_team, 1.0)
    h_def = defense_ratings.get(h_team, 1.0)
    a_att = attack_ratings.get(a_team, 1.0)
    a_def = defense_ratings.get(a_team, 1.0)
    
    home_elo_series.append(h_elo)
    away_elo_series.append(a_elo)
    home_att_series.append(h_att)
    away_att_series.append(a_att)
    home_def_series.append(h_def)
    away_def_series.append(a_def)

    # --- Actualizacion Elo ---
    # HGA: Home Ground Advantage (aprox 70 puntos)
    dr = a_elo - (h_elo + 70)
    e_prob = 1 / (1 + 10 ** (dr / 400))
    actual_score = get_result_score(row['FTR'])
    
    new_h_elo = h_elo + elo_k * (actual_score - e_prob)
    new_a_elo = a_elo + elo_k * ((1 - actual_score) - (1 - e_prob))
    
    elo_ratings[h_team] = new_h_elo
    elo_ratings[a_team] = new_a_elo
    
    # --- Actualizacion Dixon-Coles (Simplificada) ---
    # Basada en Goles Esperados vs Reales
    fthg = row['FTHG']
    ftag = row['FTAG']
    
    # Prediccion simple lambda = att * def_rival
    # Error = Real - Predicho
    # Update Att = Att + lr * Error * Def_Rival
    # Update Def = Def + lr * Error * Att_Rival (Defensa inversa a mayor rating mejor? No, usaremos multiplicador directo de goles)
    # Nota: Dixon Coles real requiere optimizacion de verosimilitud, aqui usamos un gradiente estocastico simple para adaptarlo partido a partido.
    
    if not np.isnan(fthg) and not np.isnan(ftag):
        pred_hg = h_att * a_def
        pred_ag = a_att * h_def
        
        err_h = fthg - pred_hg
        err_a = ftag - pred_ag
        
        # Actualizar ratings
        attack_ratings[h_team] += dc_lr * err_h * a_def
        defense_ratings[a_team] += dc_lr * err_h * h_att
        
        attack_ratings[a_team] += dc_lr * err_a * h_def
        defense_ratings[h_team] += dc_lr * err_a * a_att
        
        # Normalizar para evitar explosion
        # (Idealmente mantener medias en 1.0)
        

df['Home_Elo'] = home_elo_series
df['Away_Elo'] = away_elo_series
df['Home_Att_Strength'] = home_att_series
df['Away_Att_Strength'] = away_att_series
df['Home_Def_Weakness'] = home_def_series # Nota: Aqui mas alto significa que CONCEDE mas goles (Weakness)
df['Away_Def_Weakness'] = away_def_series

## 5. Exportación de Features

In [39]:
# Seleccionar columnas finales
feature_cols = [
    'Date', 'Season', 'HomeTeam', 'AwayTeam', 'FTR', 'FTHG', 'FTAG',
    'Home_Elo', 'Away_Elo',
    'Home_Att_Strength', 'Away_Att_Strength',
    'Home_Def_Weakness', 'Away_Def_Weakness',
    'Home_FIFA_Ova', 'Away_FIFA_Ova',
    'Home_Market_Value', 'Away_Market_Value',
    'Home_Rest_Days', 'Away_Rest_Days',
    'Home_xG_Proxy', 'Away_xG_Proxy',
    'Home_Dominance', 'Away_Dominance',
    'Home_Pressure', 'Away_Pressure',
    'B365H', 'B365D', 'B365A' # Cuotas para calcular ROI despues
]

# Filtrar solo lo necesario y guardar
df_final = df[feature_cols].copy()

# Limpieza final de NAs generados por proxies
df_final = df_final.ffill().fillna(0)

df_final.to_csv(OUTPUT_FILE, index=False)
print(f"✅ Features exportadas a {OUTPUT_FILE} con {len(df_final)} registros.")

✅ Features exportadas a df_final_features.csv con 5700 registros.
